In [4]:
import numpy as np
import matplotlib.pyplot as plt
from deps import cv_core
import cv2
import glob
from deps.dobot_api import DobotApiDashboard, DobotApiMove
import imutils

In [5]:
camera_res = {
            '240':(320,240),
            '480':(640,480),
            '600':(800,600),
            '768':(1024, 768),
            '960':(1280, 960),
            '1200':(1600, 1200),
            '1536':(2048, 1536),
            '1944':(2592, 1944),
            '2448':(3264, 2448)}

In [16]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['1200'])
cv_core.video_test(cap)
cap.release()

In [3]:
features_mm_to_pixels_dict =  {(382.76, -113.37): (499, 412), #bottom left
                                (225.27, 94.68): (240, 103), #top right
                                (386.5, 91.55): (492, 98), # bottom right
                                (221.25, -110.62): (248, 419)} # top left

tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [20]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['768'])
#out = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (640,480))

while(True):
    ret, frame = cap.read()

    a,b,r,sorted_contours = cv_core.find_contours(frame)
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    cv2.circle(plot_img, (a, b), 1, (255, 0, 0), 3)
    #with_contours = cv2.drawContours(plot_img, sorted_contours, -1,(0,255, 0),2)

    for c in sorted_contours:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 1, (255, 0, 0), -1)
        cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    #out.write(with_contours)

    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

In [14]:
images = sorted(glob.glob('camera_data/raw/*.png'))


font = cv2.FONT_HERSHEY_SIMPLEX
  
# org
org = (50, 50)
  
# fontScale
fontScale = 1
   
# Blue color in BGR
color = (0, 255, 0)
  
# Line thickness of 2 px
thickness = 2

for idx, image in enumerate(images):
    img = cv2.imread(image)
    img = cv2.putText(img, f'{idx}', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
    cv2.imwrite(f'camera_data/marked/image{idx}.png', img)

In [20]:
cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['1200'])

while True:
    ret, frame = cap.read()
    
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
cap.release()
cv2.destroyAllWindows()

In [4]:
img = cv2.imread('image0.png')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (9,7), None)

In [5]:
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)

In [7]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [9]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [8]:
Z = -145
for corner in corners:
    x = corner[0][0]
    y = corner[0][1]

    X, Y, _ = tf_mtx @ (x,y,1)    

    move.MovL(X, Y, Z, 0)